# **Bachelor's Thesis Notebook**

This is a notebook containing a part of the analysis of my bachelor's thesis.


In [53]:
import pandas as pd, numpy as np
import statistics

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
import warnings
warnings.filterwarnings('ignore')

le paroline tra le colonne saranno estratte co LDA somehow

## Berger et al. 2020 Models

First of all, let's reproduce their results using their data. Later on, i will replicate their analysis in order to be able to generalize the approach to new data.

In [56]:
df = pd.read_csv("/content/drive/MyDrive/Bachelor's Thesis/Files/You_S1_Data_NoBillboardRanking.csv", encoding = "ISO-8859-1")
df.head(5)

,uniq_id,date,month,year,d2014,d2015,d2016,q1_14,q2_14,q3_14,...,way,well,were,what,when,where,who,will,with,yeah
0,22,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,28,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,40,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,41,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
4,44,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


The original dataset is **NOT** sorted by id (see "*uniq_id*" column)! This needs to be fixed since the response variable (here "*Song Rank*") is missing, and it has to be built from scratch.

Thankfully, once sorted by "*uniq_id*", the 4200 songs will preserve the same ordering they had on the various ranking charts of *Billboard.com*, and thus it is enough to enumerate each sequential block of 50 rows/songs with numbers decreasing from 50 to 1 (after the reverse coding describe in Section 4.1 of my thesis).

The songs in fact were scraped from 84 different digital song sales charts, belonging to 7 major genres (ie. christian, country, dance, rock, pop, rap, r&b) and different time periods (4 times a year for 3 years).

In [57]:
df = df.sort_values(by=['uniq_id'])
df = df.reset_index(drop=True)
df.head(5)

,uniq_id,date,month,year,d2014,d2015,d2016,q1_14,q2_14,q3_14,...,way,well,were,what,when,where,who,will,with,yeah
0,1,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
1,2,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,3,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
3,4,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,5,2014-02-08,2,2014,1,0,0,1,0,0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0


In [58]:
#Build Song Rank Column
df["Rank"] = np.tile(np.arange(50, 0, -1), 84)

In [59]:
df[["uniq_id","Rank"]].head(5)

,uniq_id,Rank
0,1,50
1,2,49
2,3,48
3,4,47
4,5,46


### Model 1

In [60]:
import statsmodels.formula.api as smf
results = smf.ols('Rank ~ you', data = df).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     11.21
Date:                Fri, 02 Dec 2022   Prob (F-statistic):           0.000822
Time:                        23:00:14   Log-Likelihood:                -17165.
No. Observations:                4200   AIC:                         3.433e+04
Df Residuals:                    4198   BIC:                         3.435e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     24.5217      0.367     66.768      0.0

### Model 2

In [61]:
#Build "times charted" column
df["times_charted"] = 0
d = df.groupby(['artist_song'])['artist_song'].count().to_dict() 
for i in range(len(df.artist_song)-1):
  df.loc[i, "times_charted"] = d[df.artist_song[i]]

results = smf.ols('Rank ~ you + times_charted + count_genres + d_airplaysong', data=df).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     127.4
Date:                Fri, 02 Dec 2022   Prob (F-statistic):          7.84e-103
Time:                        23:00:15   Log-Likelihood:                -16930.
No. Observations:                4200   AIC:                         3.387e+04
Df Residuals:                    4195   BIC:                         3.390e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        14.9353      0.732     20.389

### Model 3

In [62]:
results = smf.mixedlm("Rank ~ you", df, groups=df["artist_song"]).fit().summary()
print(results)

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Rank       
No. Observations: 4200    Method:             REML       
No. Groups:       1885    Scale:              168.4553   
Min. group size:  1       Log-Likelihood:     -17085.5706
Max. group size:  20      Converged:          Yes        
Mean group size:  2.2                                    
----------------------------------------------------------
           Coef.   Std.Err.    z     P>|z|  [0.025  0.975]
----------------------------------------------------------
Intercept  24.076     0.438  54.937  0.000  23.217  24.935
you         0.143     0.066   2.158  0.031   0.013   0.272
Group Var  38.564     0.357                               



### Model 4

In [63]:
import statsmodels.api as sm
my_cols = ["you","g_dan","g_cou","g_chr","g_rap","g_pop","g_roc","g_rnb"]
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.655
Date:                Fri, 02 Dec 2022   Prob (F-statistic):              0.115
Time:                        23:00:24   Log-Likelihood:                -17165.
No. Observations:                4200   AIC:                         3.435e+04
Df Residuals:                    4192   BIC:                         3.440e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         21.4262      0.325     65.896      0.0

### Model 5

In [64]:
my_cols = ["you",'q1_14','q2_14','q3_14','q4_14','q1_15','q2_15','q3_15','q4_15','q1_16','q2_16','q3_16','q4_16']
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9338
Date:                Fri, 02 Dec 2022   Prob (F-statistic):              0.512
Time:                        23:00:24   Log-Likelihood:                -17165.
No. Observations:                4200   AIC:                         3.436e+04
Df Residuals:                    4187   BIC:                         3.444e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         22.6331      0.340     66.620      0.0

### Model 6

In [65]:
my_cols = ["you"] + ["lda10topic{}".format(x) for x in range(1,11)] 
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.306
Date:                Fri, 02 Dec 2022   Prob (F-statistic):            0.00818
Time:                        23:00:25   Log-Likelihood:                -17158.
No. Observations:                4200   AIC:                         3.434e+04
Df Residuals:                    4188   BIC:                         3.442e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -19.3327    124.495     -0.155   

### Model 7

In [66]:
my_cols = ["you","cogproc","affect1","percept","drives","time","relativ","informal"] 
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     3.120
Date:                Fri, 02 Dec 2022   Prob (F-statistic):            0.00162
Time:                        23:00:25   Log-Likelihood:                -17158.
No. Observations:                4200   AIC:                         3.433e+04
Df Residuals:                    4191   BIC:                         3.439e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         25.1366      1.148     21.888      0.0

### Model 8

In [67]:
my_cols = list(df.columns)[64:-2] + ["you"]
df=df.fillna(0) #??????????
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     2.092
Date:                Fri, 02 Dec 2022   Prob (F-statistic):           3.01e-09
Time:                        23:00:25   Log-Likelihood:                -17068.
No. Observations:                4200   AIC:                         3.433e+04
Df Residuals:                    4101   BIC:                         3.496e+04
Df Model:                          98                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         28.6852      0.764     37.541      0.0

### Model 9

In [68]:
df["Intercept"]=1
my_cols =  ["times_charted","count_genres","d_airplaysong"]+list(df.columns)[64:-2]+["cogproc","affect1","percept","drives","time","relativ","informal"]+["lda10topic{}".format(x) for x in range(1,11)]+['q1_14', 'q2_14','q3_14','q4_14','q1_15','q2_15','q3_15','q4_15','q1_16','q2_16','q3_16','q4_16']+["g_dan","g_cou","g_chr","g_rap","g_pop","g_roc","g_rnb"]
md = sm.MixedLM(df.Rank, df[my_cols + ["you","Intercept"]].drop(columns=["Rank"]), groups=df["artist_song"])#, exog_re=df["Intercept"])
mdf = md.fit()
print(mdf.summary())

                 Mixed Linear Model Regression Results
Model:                 MixedLM      Dependent Variable:      Rank       
No. Observations:      4200         Method:                  REML       
No. Groups:            1885         Scale:                   140.1407   
Min. group size:       1            Log-Likelihood:          -16615.8785
Max. group size:       20           Converged:               No         
Mean group size:       2.2                                              
------------------------------------------------------------------------
               Coef.     Std.Err.    z    P>|z|    [0.025       0.975]  
------------------------------------------------------------------------
times_charted    0.885       0.118  7.517 0.000        0.654       1.115
count_genres     6.115       0.796  7.678 0.000        4.554       7.675
d_airplaysong   11.604       0.575 20.164 0.000       10.476      12.732
about            0.305       0.695  0.439 0.660       -1.057       1.

## Berger et al. 2018 Models

LSM stuff...

In [69]:
songTopicComp = []
for x in range(len(df.uniq_id)):
    songTopicComp.append(list(df[["lda10topic{}".format(x) for x in range(1,11)]].iloc[x]))

genres = ["Dan", "Pop", "Rap", "RnB", "Roc", "Chr", "Cou"]
dic_genres = {"Dan":0, "Pop":1, "Rap":2, "RnB":3, "Roc":4, "Chr":5, "Cou":6}
avgTopComp = []

for x in genres:
    aa = [songTopicComp[y] for y in range(len(df.genre)) if df.genre[y]==x]
    temp=[]
    for y in range(10):
        temp.append(statistics.mean([aa[z][y] for z in range(len(aa))]))
    avgTopComp.append(temp)

lsm = []
for x in range(len(df.genre)):
    lsm.append(
        statistics.mean(list(np.divide(abs(np.array(songTopicComp[x]) - np.array(avgTopComp[dic_genres[df.genre[x]]])),
                           np.array(songTopicComp[x]) + np.array(avgTopComp[dic_genres[df.genre[x]]]) + 0.0001)))
    )
df["lsm"] = lsm

### Model 1

In [70]:
import statsmodels.formula.api as smf
results = smf.ols('Rank ~ lsm', data=df).fit().summary()
print(results)

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     9.270
Date:                Fri, 02 Dec 2022   Prob (F-statistic):            0.00234
Time:                        23:02:12   Log-Likelihood:                -17166.
No. Observations:                4200   AIC:                         3.434e+04
Df Residuals:                    4198   BIC:                         3.435e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     21.4483      1.349     15.897      0.0

### Model 2

In [71]:
import re
a = list(df.artist)
processed=[]
exclusions = '|'.join([" with ", " featuring ", " feat ", " & ", ", ", " ft. ", " ft ", " and ", " \+ ", " x "])
for x in a:
    x = re.sub(exclusions, "|", x.lower())
    x = re.sub("  ", "", x)
    processed.append(x.split("|"))
totalCols = []
for x in processed:
    print(x)
    for y in x:
        totalCols.append(y)
totalCols = set(totalCols)
oldCols = len(list(df.columns))
for x in totalCols:
    #print(x)
    df[x] = 0
for x in range(len(df.artist)):
    for y in processed[x]:
        df[y][x] = 1
artists = list(df.columns)[oldCols:]

time = ['q1_14', 'q2_14','q3_14','q4_14','q1_15','q2_15','q3_15','q4_15','q1_16','q2_16','q3_16','q4_16']

date_dic = {'q1_14':1, 'q2_14':2,'q3_14':3, 'q4_14':4, 'q1_15':5, 'q2_15':6, 'q3_15':7, 'q4_15':8, 'q1_16':9, 'q2_16':10, 'q3_16':11, 'q4_16':12}
df["date"] = 0
for x in list(date_dic.keys()):
    for y in range(len(df.genre)):
        if df[x][y] == 1:
            df["date"][y] = date_dic[x]

my_cols = ["lsm", "d_airplaysong", "count_genres", "times_charted"] + time + artists + ["lda10topic{}".format(x) for x in range(1,11)]


['hillsong united']
['jamie grace']
['mandisa']
['phil wickham']
['mercyme']
['francesca battistelli']
['matt redman']
['matt maher']
['the fray']
['casting crowns']
['sidewalk prophets']
['mercyme']
['chris tomlin']
['switchfoot']
['crowder']
['skillet']
['the fray']
['casting crowns']
['big daddy weave']
['needtobreathe']
['needtobreathe']
['carrie underwood']
['newsboys']
['matthew west']
['tobymac']
['big daddy weave']
['mandisa', 'tobymac']
['skillet']
['britt nicole']
['the afters']
['skillet']
['casting crowns']
['tenth avenue north']
['lifehouse']
['chris tomlin']
['paul van dyk', 'plumb']
['skillet']
['jj weeks band']
['matt redman']
['carrie underwood']
['switchfoot']
['casting crowns']
['chris tomlin']
['joni eareckson tada']
['rend collective experiment']
['hillsong']
['unspoken']
['passion', 'kristian stanfill']
['laura story']
['7eventh time down']
['luke bryan']
['rascal flatts']
['brantley gilbert']
['david nail']
['cole swindell']
['eric church']
['hunter hayes']
['jas

In [72]:
import statsmodels.api as sm
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.416
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     2.336
Date:                Fri, 02 Dec 2022   Prob (F-statistic):           2.14e-69
Time:                        23:02:45   Log-Likelihood:                -16042.
No. Observations:                4200   AIC:                         3.405e+04
Df Residuals:                    3218   BIC:                         4.028e+04
Df Model:                         981                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   

### Model 3

KeyError: "['affect', 'social', 'swear'] not in index"

In [73]:
my_cols = ["lsm", "d_airplaysong", "count_genres", "times_charted",] + time  + ["lda10topic{}".format(x) for x in range(1,11)] + ["WC", "cogproc", "percept", "drives", "relativ", "time"] +artists

import statsmodels.api as sm
X = sm.add_constant(df[my_cols])
results = sm.OLS(df.Rank, X).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                   Rank   R-squared:                       0.417
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     2.327
Date:                Fri, 02 Dec 2022   Prob (F-statistic):           6.14e-69
Time:                        23:02:49   Log-Likelihood:                -16038.
No. Observations:                4200   AIC:                         3.405e+04
Df Residuals:                    3212   BIC:                         4.032e+04
Df Model:                         987                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   